In [ ]:
# 학습한 모델을 기준으로 다음 년도(2024) 가격 예측 및 매수/매도 시점 예측하기

In [81]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 1. OHLCV 데이터 및 기술적 지표 로드
data = pd.read_csv('/home/proj/finTF/store_data/process/OHLCV_info/calculate_total_year/건설업.csv')
df = pd.DataFrame(data)
df['날짜'] = pd.to_datetime(df['날짜'])

# 2. 특성과 타겟 설정 (예를 들어, 종가를 예측하려고 할 때)
X = df[['시가', '고가', '저가', '거래량', 'MA_20', 'RSI_14', 'BBP_20_2.0']]
y = df['종가']  # 종가를 예측

# 3. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 랜덤 포레스트 회귀 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5. 미래 데이터 예측
future_prices = model.predict(X_test)

# 6. 예측된 미래 날짜 생성 (2024년 영업일)
future_dates = pd.bdate_range(start='2024-01-01', periods=len(future_prices))

# 7. 예측 결과를 데이터프레임으로 저장
df_future = pd.DataFrame(future_prices, columns=['예측된 종가'])
df_future['날짜'] = future_dates  # 2024년의 영업일을 예측된 종가와 매핑

# 8. 예측된 데이터를 바탕으로 기술적 지표 계산

# 이동평균 계산
df_future['MA_20'] = df_future['예측된 종가'].rolling(window=20).mean()

# RSI 계산
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window).mean()
    avg_loss = pd.Series(loss).rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df_future['RSI_14'] = calculate_rsi(df_future['예측된 종가'], window=14)

# 볼린저 밴드 계산
df_future['BBM_20_2.0'] = df_future['예측된 종가'].rolling(window=20).mean()  # 중간선
df_future['BBU_20_2.0'] = df_future['BBM_20_2.0'] + 2 * df_future['예측된 종가'].rolling(window=20).std()  # 상단선
df_future['BBL_20_2.0'] = df_future['BBM_20_2.0'] - 2 * df_future['예측된 종가'].rolling(window=20).std()  # 하단선

# 9. 매수/매도 여부 판단
df_future['매수매도'] = df_future['RSI_14'].apply(lambda x: '매수' if x < 30 else ('매도' if x > 70 else '유지'))

# 10. 예측된 미래 데이터 출력
print(df_future[['날짜', '예측된 종가', 'RSI_14', 'BBU_20_2.0', 'BBL_20_2.0', '매수매도']])


            날짜    예측된 종가     RSI_14  BBU_20_2.0  BBL_20_2.0 매수매도
0   2024-01-01   88.1866        NaN         NaN         NaN   유지
1   2024-01-02  112.7480        NaN         NaN         NaN   유지
2   2024-01-03  121.3618        NaN         NaN         NaN   유지
3   2024-01-04  110.8272        NaN         NaN         NaN   유지
4   2024-01-05   80.5591        NaN         NaN         NaN   유지
..         ...       ...        ...         ...         ...  ...
242 2024-12-04  114.2770  53.881767  145.461198   62.678992   유지
243 2024-12-05  120.3404  57.419032  147.105712   63.644058   유지
244 2024-12-06   85.9943  43.140658  147.313153   62.652517   유지
245 2024-12-09  134.7884  50.867079  149.069047   67.446103   유지
246 2024-12-10  138.6449  53.511894  151.770923   70.703927   유지

[247 rows x 6 columns]


In [ ]:
# 최적화 모델 적용하기

In [83]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 1. OHLCV 데이터 및 기술적 지표 로드
data = pd.read_csv('/home/proj/finTF/store_data/process/OHLCV_info/calculate_total_year/건설업.csv')
df = pd.DataFrame(data)
df['날짜'] = pd.to_datetime(df['날짜'])

# 2. 특성과 타겟 설정 (예를 들어, 종가를 예측하려고 할 때)
X = df[['시가', '고가', '저가', '거래량', 'MA_20', 'RSI_14', 'BBP_20_2.0']]
y = df['종가']  # 종가를 예측

# 3. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 랜덤 포레스트 회귀 모델 학습
model = RandomForestRegressor(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=800,
    random_state=42
)

# 모델 학습
model.fit(X_train, y_train)

# 5. 미래 데이터 예측
future_prices = model.predict(X_test)

# 6. 예측된 미래 날짜 생성 (2024년 영업일)
future_dates = pd.bdate_range(start='2024-01-01', periods=len(future_prices))

# 7. 예측 결과를 데이터프레임으로 저장
df_future = pd.DataFrame(future_prices, columns=['예측된 종가'])
df_future['날짜'] = future_dates  # 2024년의 영업일을 예측된 종가와 매핑

# 8. 예측된 데이터를 바탕으로 기술적 지표 계산

# 이동평균 계산
df_future['MA_20'] = df_future['예측된 종가'].rolling(window=20).mean()

# RSI 계산
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window).mean()
    avg_loss = pd.Series(loss).rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df_future['RSI_14'] = calculate_rsi(df_future['예측된 종가'], window=14)

# 볼린저 밴드 계산
df_future['BBM_20_2.0'] = df_future['예측된 종가'].rolling(window=20).mean()  # 중간선
df_future['BBU_20_2.0'] = df_future['BBM_20_2.0'] + 2 * df_future['예측된 종가'].rolling(window=20).std()  # 상단선
df_future['BBL_20_2.0'] = df_future['BBM_20_2.0'] - 2 * df_future['예측된 종가'].rolling(window=20).std()  # 하단선

# 9. 매수/매도 여부 판단
df_future['매수매도'] = df_future['RSI_14'].apply(lambda x: '매수' if x < 30 else ('매도' if x > 70 else '유지'))

# 10. 예측된 미래 데이터 출력
print(df_future[['날짜', '예측된 종가', 'RSI_14', 'BBU_20_2.0', 'BBL_20_2.0', '매수매도']])


            날짜      예측된 종가     RSI_14  BBU_20_2.0  BBL_20_2.0 매수매도
0   2024-01-01   88.032761        NaN         NaN         NaN   유지
1   2024-01-02  113.037480        NaN         NaN         NaN   유지
2   2024-01-03  121.706002        NaN         NaN         NaN   유지
3   2024-01-04  110.371253        NaN         NaN         NaN   유지
4   2024-01-05   80.660263        NaN         NaN         NaN   유지
..         ...         ...        ...         ...         ...  ...
242 2024-12-04  114.281535  53.930473  145.407950   62.637510   유지
243 2024-12-05  120.039266  57.393173  147.021032   63.604958   유지
244 2024-12-06   85.888031  43.087296  147.231636   62.588587   유지
245 2024-12-09  134.858324  50.840293  149.024108   67.353093   유지
246 2024-12-10  138.432723  53.463660  151.702775   70.616292   유지

[247 rows x 6 columns]
